####25 May 2015

##Exercises In Computation

###Discussion Notes I

###Elements of a Programming Language

* Primitive Expressions
* Means of Combination
* Means of Abstraction

####Consider a procedure to compute the factorial of a number n

In [60]:
function Rfactorial(n)
    if n == 1
        n
        else n*Rfactorial(n-1)
    end
end

Rfactorial (generic function with 1 method)

In [61]:
Rfactorial(3)

6

In [62]:
Rfactorial(10)

3628800

* Distinguish special forms of the language (e.g., `function`) that have pre-defined behavior at evaluation--distinguish these from expressions that you write and that Julia evaluates according to rules we will learn.

Rfactorial(n) defines a recursive procedure. Here is an iterative procedure (still a recursive process).

In [63]:
function Ifactorial(n)
    function Iterator(product, counter)
        if counter == 1
            product
            else Iterator((counter*product), (counter - 1))
        end
    end
    Iterator(1, n)
end

Ifactorial (generic function with 1 method)

In [64]:
Ifactorial(3)

6

In [65]:
Ifactorial(10)

3628800

* Julia NB: the comma separating the arguments in the function `Iterator`.

###Naming and the Environment

Attach a name to a value with assignment using `=`

In [16]:
ϕ = 8

8

In [17]:
ϕ

8

Julia NB: type `\phi + tab` for phi (other unicode characters similarly). The Julia docs have a long list of characters.

In [18]:
ϕ / 2

4.0

###Compound Procedures

In [66]:
function square(x)
    x*x
end

square (generic function with 1 method)

In [67]:
square(5)

25

In [68]:
function sumSquares(x, y)
    square(x) + square(y)
end

sumSquares (generic function with 1 method)

In [24]:
sumSquares(ϕ, 5)

89

In [69]:
function aPlusAbsb(a, b)    #  subtract b from a if b<0, otherwise add b to a 
    function op(s)
        if s < 0
            -
        else
            +
        end
    end
    apply(op(b), a, b)
end
        

aPlusAbsb (generic function with 1 method)

In [70]:
aPlusAbsb(0, -5)

5

In [71]:
aPlusAbsb(5, 5)

10

###Example: compute the square root of x using Newton's method

First test whether a guess is good enough.  

A guess is good enough if the square of the guess is within a specified neighborhood of x.  

Return guess if it is good enough.  

Otherwise try a new guess equal to the average
> (x + x/guess) / 2

In [72]:
function sqrtIter(guess, x)
    function average(x, y)  # compute average of x and y
        return (x+y)/2
    end
    function goodEnough(guess, x)  # goodEnough tests whether the guess is close enough to x
        return abs(guess - x/guess)/guess < 0.01
    end
    function improve(guess, x)  # improve guess with average of guess and x/guess
        return average(guess, x/guess)
    end
    if goodEnough(guess, x)
        guess
    else
        sqrtIter(improve(guess, x), x)
    end
end

function SquareRoot(x)
    return sqrtIter(1.0, x)
end

SquareRoot (generic function with 1 method)

####Notes on Computation Structure

* `sqrtIter` illustrates a computation structure based on the conceptual steps necessary to effect the computation. One way to think of this structure is simply to focus on the ideas required for the computation and to write the main computation procedure assuming that you have the functions that give effect to those ideas. Once you have a sketch procedure that reaches the result by way of this *wishful thinking* then you can go back and write the procedures you need (here, for example, `average`, `goodEnough`, `improve`). Structuring a computation this way highlights our understanding of the computation process and helps us evaluate that understanding and consider alternatives.


* Name spaces. If you will not generally need procedures such as `goodEnough` outside of your main computation, then it makes sense to write it internally to the main procedure as we did in `sqrtIter`. This way the names are used only locally to our `sqrtIter`.

In [73]:
SquareRoot(144)

12.000545730742438

###Example: Explore Computing of Exponentials

Julia has an operator *`x^y`* to compute x to the power of y, but let's look a few ways to write a power function ourselves.

Computing b<sup>n</sup> lends itself to a recursive process:
>b<sup>n</sup>  =  b \* b<sup>n-1</sup>, with the base case

>b<sup>0</sup>  =  1

In [74]:
function PowerR(b, n)
    if n == 0
        1
    else
        b*PowerR(b, n-1)
    end
end

PowerR (generic function with 1 method)

In [75]:
PowerR(2, 4)

16

In [76]:
PowerR(2, 5)

32

In [77]:
PowerR(34, 65)

0

In [78]:
34^65

0

In [79]:
PowerR(big(34), big(65))

3516653733747887484905571395749820949396267082812326251249703698546110535162463532902715566036353024

In [80]:
big(34^65)

0

In [81]:
big(34)^big(65)

3516653733747887484905571395749820949396267082812326251249703698546110535162463532902715566036353024

A power function described by an iterative process:

In [82]:
function PowerI(b, n)
    function iterator(n, comp)  # iterator will accumulate the computation at each step
        if n == 0
            comp
        else
            iterator(n-1, b*comp)
        end
    end
    iterator(n, 1)
end
            

PowerI (generic function with 1 method)

In [83]:
PowerI(2, 8)

256

In [84]:
PowerI(big(34), big(65))

3516653733747887484905571395749820949396267082812326251249703698546110535162463532902715566036353024

If n is even then 
>b<sup>n</sup>  =  (b<sup>n/2</sup>)<sup>2</sup>  =  (b<sup>n/2</sup>) * (b<sup>n/2</sup>)

Using this simple property reduces the number of computations exponentially.

In [85]:
function PowerF(b, n)
    function even(j)   # even tests whether j is an even number
        j%2 == 0       # % is Julia's modulo operator
    end
    function square(x)  # square returns x^2
        x*x
    end
    if n == 0
        1
    elseif even(n)
        square(PowerF(b, n/2))
    else b*PowerF(b, n-1)
    end     
end

PowerF (generic function with 1 method)

In [86]:
PowerF(2, 10)

1024

In [91]:
@time PowerF(2, 10)

elapsed time: 2.412e-5 seconds (4960 bytes allocated)


1024

In [94]:
@time PowerI(2, 10)

elapsed time: 1.7574e-5 seconds (704 bytes allocated)


1024

In [97]:
@time PowerR(2, 10)

elapsed time: 6.081e-6 seconds (96 bytes allocated)


1024

In [115]:
@time PowerF(big(21), big(32))

elapsed time: 0.000216602 seconds (8888 bytes allocated)


2046526777500669368329342638102622164679041

In [116]:
@time PowerI(big(21), big(32))

elapsed time: 0.000534246 seconds (4624 bytes allocated)


2046526777500669368329342638102622164679041

In [117]:
@time PowerR(big(21), big(32))

elapsed time: 0.000290597 seconds (3912 bytes allocated)


2046526777500669368329342638102622164679041

In [118]:
function PowerF2(b, n)
    if n%2 == 0
        PowerF2(b, n/2)*PowerF2(b, n/2)
        else 
        b*PowerF(b, n-1)
    end     
end

PowerF2 (generic function with 1 method)

In [123]:
@time PowerF2(big(21), big(32))

elapsed time: 0.000927549 seconds (45240 bytes allocated)


2046526777500669368329342638102622164679041

In [124]:
PowerF2(2, 54)

18014398509481984

In [127]:
@time PowerF2(2, 54)

elapsed time: 7.1684e-5 seconds (13440 bytes allocated)


18014398509481984

In [129]:
@time PowerI(2, 54)

elapsed time: 2.6838e-5 seconds (1408 bytes allocated)


18014398509481984

In [131]:
@time PowerR(2, 54)

elapsed time: 6.251e-6 seconds (96 bytes allocated)


18014398509481984

NB. The recursive definition is fastest and takes least space suggesting tail recursive implementation. Explain what is going on.

###Example Computation: Greatest Common Divisor

In [63]:
gcd(24, 16)

8

####Euclid's Method of Computing the GCD of Two Numbers

If r is the remainder of a/b then
>GCD(a,b)  = GCD(b, r)

In [132]:
function EuclidGCD(a, b)
    if b == 0 
        a
    else
        EuclidGCD(b, a%b)   # a%b returns the remainder of a/b
    end
end

EuclidGCD (generic function with 1 method)

In [2]:
EuclidGCD(34325, 3433)

1

In [3]:
gcd(34325, 3433)

1

In [4]:
gcd(144, 24)

24

In [5]:
EuclidGCD(144, 24)

24

In [6]:
@time gcd(2455, 55)

elapsed time: 0.001062587 seconds (13880 bytes allocated)


5

In [7]:
@time EuclidGCD(2455, 55)

elapsed time: 5.799e-6 seconds (80 bytes allocated)


5

Why is EuclidGCD faster than built-in gcd?

###Example: Testing for Primality

In [133]:
function SmallestDivisor(n) # returns the smallest divisor of integer n
    function FindDivisor(n, testdivisor)
        if testdivisor^2 > n
            n
            elseif n%testdivisor == 0
            testdivisor
        else
            FindDivisor(n, testdivisor+1)
        end
    end
    FindDivisor(n, 2)
end

function IsPrime(n) # returns True if n is prime, false otherwise
    SmallestDivisor(n) == n
end

IsPrime (generic function with 1 method)

In [9]:
IsPrime(6)

false

In [10]:
IsPrime(5)

true

In [11]:
IsPrime(7)

true

In [12]:
isprime(4)

false

In [13]:
isprime == IsPrime

false

It turns out that Julia's primality test is also called isprime but that Julia variable names are case-sensitive so isprime is not the same as IsPrime.

In [135]:
@time isprime(546434543)

elapsed time: 1.9975e-5 seconds (200 bytes allocated)


false

In [137]:
@time IsPrime(546434543)

elapsed time: 5.3845e-5 seconds (696 bytes allocated)


false

###Congruence Modulo n

Integers a and b are congruent modulo n if they both have the same remainder when divided by n.

####Fermat's Little Theorem

If n is a prime number and a is any positive integer less than n, then a raised to the nth power is congruent to a modulo n.

Take n = 7

Say a = 2

>a%n =  a (where % is the modulo operator)

>2%7 = 2

>note that a%n = a when a < n

So Fermat says that

> 2<sup>7</sup> % 7 = 2

> 2 &times; 2 &times; 2 &times; 2 &times; 2 &times; 2 &times; 2 % 7 = 2

Prove this: n prime, a < n &Rightarrow; a<sup>n</sup>/n  =  n*x + a, for some integer x



In [138]:
function expmod(base, exp, m)  # expmod computes base^exp modulo m
    if exp == 0
        1
        elseif exp%2 == 0
        (expmod(base, exp/2, m)^2)%m
    else
        (base*expmod(base, exp-1, m))%m
    end
end

expmod (generic function with 1 method)

####Fermat's Test for Primality (probabilistic)

Testing whether n is prime:

Choose a random integer, a < n

Check whether *a<sup>n</sup> modulo n  =  a*

If no, then n is certainly not prime

If yes, then run test again with a new integer value for a.


In [139]:
function FermatTest(n)  # returns true if n is certainly prime, false if not sure
    a = rand(1:n)  # rand(1:n) returns a random natural number < n
    expmod(a, n, n) == a  
end

FermatTest (generic function with 1 method)

In [38]:
FermatTest(17)

true

In [39]:
FermatTest(18)

false

In [40]:
FermatTest(4)

false

In [140]:
function randTester(n)
    for i in 1:n*n
        println(rand(1:n))
    end
end

randTester (generic function with 1 method)

In [141]:
randTester(3)

1
1
2
2
3
3
2
3
3


The next function, fastPrime takes two arguments, n and i, and runs FermatTest(n) i number of times. 

In [142]:
function fastPrime(n, i)
    if i == 0
        true
    elseif FermatTest(n)
        fastPrime(n, i-1)
    else
        false
    end
end

fastPrime (generic function with 1 method)

In [143]:
fastPrime(561, 50)

true

561 is actually not prime. It fools the Fermat test because a<sup>561</sup> modulo 561 = a but 561 is not prime.

In [49]:
561 / 3

187.0

In [56]:
big((4^561)%561)

0

In [58]:
big((11^561)%561)

137

In [45]:
fastPrime(23, 5)

true

In [46]:
fastPrime(24, 5)

false

In [48]:
fastPrime(123, 20)

false

###Using Procedures To Compute With Abstractions

###Procedures As Arguments

See SICP p. 57.

The function `sum` generalizes the addition of a sequence of terms formed from the integers in the interval `[a, b]`.

* The argument `term` takes a function value that computes the the i<sup>th</sup> term of the sequence given i.

* The argument `next` takes a function value that computes the next term in the sequence given the i<sup>th</sup> term.

* The function `sum` keeps adding terms that it can form within the interval [a, b].

In [146]:
function sum(term, a, next, b)
    if a > b
        0
    else
        term(a) + sum(term, next(a), next, b)
    end
end     

sum (generic function with 1 method)

For example, let's sum up the cubes of the integers over [a, b]:

In [147]:
function sumcubes(a, b)
    function Inc(j)
        j+1
    end
    function cube(x)
        x*x*x
    end
    sum(cube, a, Inc, b)
end

sumcubes (generic function with 1 method)

In [148]:
sumcubes(1, 3)

36

In [149]:
sumcubes(1, 10)

3025

In [150]:
sumcubes(1, 8)

1296

The `sum` function allows you to define terms by way of the `term` function variable. Sometimes we simply want the terms to be the integers from a to b. For this we can define an identity function. 

In [158]:
function identity(x)
    x
end

identity (generic function with 2 methods)

In [159]:
function sumintegers(a, b)
    sum(identity, a, Inc, b)
end

sumintegers (generic function with 1 method)

In [160]:
function Inc(i)
    i+1
end

Inc (generic function with 1 method)

In [161]:
sumintegers(1, 5)

15

In [162]:
sumintegers(1, 10)

55

In [163]:
sumintegers(1, 100)

5050

In [164]:
sumintegers(1, 1000)

500500

Let's use a series to compute π:

π/8  =  1/1&times;3  +  1/5&times;7  +  1/9&times;11  +  1/13&times;15  +  1/17&times;19  +  ...  (let's call this π-series)

So π is &approx; 8 &times; π-series from 1 to n for large n

In [165]:
function pisum(a, b)
    function piterm(x)
        1.0/(x*(x+2))
    end
    function pinext(x)
        x+4
    end
    sum(piterm, a, pinext, b)
end

pisum (generic function with 1 method)

In [166]:
function piseriesapprox(n)
    8*pisum(1, n)
end

piseriesapprox (generic function with 1 method)

In [167]:
piseriesapprox(10)

2.976046176046176

In [168]:
piseriesapprox(100)

3.1215946525910105

In [169]:
piseriesapprox(1000)

3.139592655589783

In [170]:
piseriesapprox(10000)

3.141392653591793

In [171]:
piseriesapprox(100000)

3.141572653589795

In [179]:
piseriesapprox(110000)

3.1415744717716128

We can similarly define the product abstraction using a function. `product` takes 4 arguments: a and b, define the interval [a, b]; the `factor` argument is a function that takes one integer and computes the factor of the product in question; and `next`, an argument taking a function of one integer to return the next integer in the interval with which to compute the next factor.

In [180]:
function product(a, factor, next, b)
    if a > b
        1
    else
        factor(a) * product(next(a), factor, next, b)
    end
end

product (generic function with 1 method)

In [181]:
function factorialbyproduct(n)
    product(1, x->x, i->i+1, n)
end

factorialbyproduct (generic function with 1 method)

In [182]:
factorialbyproduct(3)

6

In [183]:
factorialbyproduct(5)

120

In [184]:
factorialbyproduct(10)

3628800

Here is another series of n terms to approximate π for large n:

>π/4  = (2/3) &times; (4/3) &times; (4/5) &times; (6/5) &times; (6/7) &times; (8/7) ... (n terms)

The RHS is equal to the product of factors,

####the i<sup>th</sup> factor:

[ (i-1) &times; (i+1) ] / (i &times; i) for i odd beginning with 3
 

In [221]:
function approxpiproduct(n)
    function plus2(i)
        i+2
    end
    function apfactor(i)
        ((i-1)*(i+1))/(i*i)
    end
    4*product(3, apfactor, plus2, n)
end

approxpiproduct (generic function with 1 method)

In [222]:
approxpiproduct(1)

4

In [223]:
approxpiproduct(3)

3.5555555555555554

In [224]:
approxpiproduct(6)

3.413333333333333

In [229]:
approxpiproduct(10)

3.302393550012597

In [220]:
π

π = 3.1415926535897...

In [225]:
π - approxpiproduct(10)

-0.16080089642280404

In [226]:
π - approxpiproduct(100)

-0.015747035627771133

In [227]:
π - approxpiproduct(10000)

-0.0001570835595026132

In [228]:
approxpiproduct(10000)

3.1417497371492957

###Example: Finding roots of equations using the half-interval method

The idea is simple. A function f, crosses the x-axis for some value of x so that f changes sign as it crosses x. Our search will close in on x by tracking x values giving f values of different sign.

Let's say that we are close enough to a root if we know that it lies within an interval < 0.0001 in length.

In [231]:
function closeEnough(x, y) # closeEnough returns True if x and y are closer than 0.0001
    (abs(x-y)) < 0.0001
end

closeEnough (generic function with 1 method)

As we close in on a root of f(x), we will consider the mid-point between the end points of our interval. We compute this as the average of the end points using the `average` function defined here:

In [233]:
function average(x, y)
    (x+y)/2
end

average (generic function with 1 method)

In [234]:
function search(f, negpoint, pospoint)
    let midpoint = average(negpoint, pospoint)
        if closeEnough(negpoint, pospoint)
            midpoint
        else
            let testvalue = f(midpoint)
                if testvalue > 0
                    search(f, negpoint, midpoint)
                    elseif testvalue == 0
                    midpoint
                else
                    search(f, midpoint, pospoint)
                end
            end
        end
    end
end

search (generic function with 1 method)

In [235]:
function halfIntervalMethod(f, a, b)
    if f(a) < 0 && f(b) > 0
        search(f, a, b)
        elseif f(a) > 0 && f(b) < 0
        search(f, b, a)
        elseif f(a) == 0
        a
        elseif f(b) == 0
        b
        else error("values are not of opposite sign", a, b)
    end
end

halfIntervalMethod (generic function with 1 method)

Let's approximate π as the root of sin(x) between 2 and 4:

In [237]:
halfIntervalMethod(sin, 2, 4)

3.141571044921875

####Example: Fixed Points of Functions

Given a function f(x), a fixed point of f is a point x s.t. `f(x) = x`.

Sometimes you can find a fixed point of f from ain initial guess `g` by repeatedly appling f:

> f(f(f(f( ... f(g) ... ))))

To begin let's set a variable specifying how close we need to be to what we are looking for:

In [238]:
tolerance = 0.0000001

1.0e-7

In [240]:
function fixedpoint1(f, initialguess)
    function closeEnough(v1, v2)
        abs(v2-v1) < tolerance
    end
    function tryanother(guess)
            let next = f(guess)
                if closeEnough(guess, next)
                    next
                else
                tryanother(next)
                end
        end
    end
    tryanother(initialguess)
end

fixedpoint1 (generic function with 1 method)

Find the fixed point of f(x) = cos(x) starting with guess = 1.0

In [241]:
fixedpoint1(cos, 1.0)

0.7390851699445544

In [242]:
cos(0.7390851699445544)

0.7390851084737987

####Example: Finding the square root of x can be expressed as a fixed point search:

>y<sup>2</sup>  =  x

>y  =  x/y

>y  =  &radic;x

Consider the function g(y):
> y &longrightarrow; x/y

The square root of x is the fixed point of g(y).

The function above, fixedpoint1, will not work with g however because it would oscillate between 2 values:

>(I)  g(y)  = x/y

>(II) g(x/y) = x / (x/y)  =  x*y / x  = y

So repeated applications of g will not converge on a unique fixed point.

We can break out of the oscillation using `average damping`.

The idea is to apply g to the average of y and g(y) instead of applying g simply to g(y):

>y' = [y + x/y] / 2
>y' = [y<sup>2</sup> + x] / 2y

In [245]:
function sqrtbyfixedpoint(x)
    function fxdpt(f, iguess)
        function clsenough(v1, v2)
            abs(v2-v1) < tolerance
        end
        function average(a, b)
            (a+b)/2
        end
        function tryguess(guess)
            let next = average(guess, f(guess))
                if clsenough(guess, next)
                    next
                else
                    tryguess(next)
                end
            end
        end
        tryguess(iguess)
    end
    fxdpt(y->x/y, 1.0)
end

sqrtbyfixedpoint (generic function with 1 method)

In [246]:
sqrtbyfixedpoint(144)

12.0

In [247]:
sqrtbyfixedpoint(81)

9.0

In [249]:
sqrtbyfixedpoint(125)

11.180339887498949

####Example: Half Interval Method to solve f(x) = 0, f continuous

Given a and b with `f(a) < 0 < f(b)`, f must have a zero on [a, b]. 

To find this zero, let mid-point m = (a+b)/2.

Then if f(m) > 0, continue search in the smaller interval [a, m].

If f(m) < 0 , search [m, b].

If f(m) = 0, return m as the zero.

In [250]:
function findzero(f, a, b)
    let m = (a+b)/2
        if abs(f(m)) < 0.0001
            m
            elseif f(m) > 0
            findzero(f, a, m)
        else
            findzero(f, m, b)
        end
    end
end

findzero (generic function with 1 method)

In [251]:
findzero(cos, -3, 0)

-1.57086181640625

In [252]:
sin(-1.57086181640625)

-0.9999999978555554

The next example shows a procedure that returns a procedure.

An average damping function returns a function that is the average of f(x) and x so 

>Damping Function d

>x &longrightarrow; (x + f(x))/2

In [253]:
function damping(f)
    x-> (x+f(x))/2
end

damping (generic function with 1 method)

The square root function computed using the fixed point equation can be re-written using average damping:

In [255]:
function sqrtbyavedamp(x)
    fixedpoint1(damping(y->x/y), 1.0)
end

sqrtbyavedamp (generic function with 1 method)

In [256]:
sqrtbyavedamp(144)

12.0

We can also use damping to compute the cube root of x using fixed point.

> y  =  x<sup>3</sup>

> Isolate the identity function:

> y/x<sup>2</sup> = x.

So we search for the fixed point of 

>y &longrightarrow; y/x<sup>2</sup>

>This is a function g(x) = y/x<sup>2</sup> given a number y (for which we want the cube root).

In [264]:
function crootbyfp(y)
    fixedpoint1(damping(x->y/(x^2)), 1.0)
end

crootbyfp (generic function with 1 method)

In [265]:
crootbyfp(125)

5.0000000176704855

In [266]:
crootbyfp(27)

3.000000021624214

####Example: Newton's Method

If ge is a differentiable function then a solution to `g(x) = 0` is a fixed point of the function f such that

>f(x) = x - g(x)/Dg(x).

The derivative

>Dg(x) = [g(x+dx) - g(x)] / dx

Let's define a little dx as 0.000001:

In [267]:
dx = 0.000001

1.0e-6

In [268]:
function derivative(g)
    x->(g(x+dx)-g(x))/dx
end

derivative (generic function with 1 method)

Let's define a function for x &longrightarrow; x<sup>3</sup> :

In [282]:
function cube(x)
    x*x*x
end

cube (generic function with 1 method)

In [283]:
cube(5)

125

In [284]:
derivative(cube)(5)

75.00001501625775

Now with the derivative function we can build the Newton transform:

In [285]:
function Newtontransform(g)
    x -> x - g(x)/derivative(g)(x)
end

Newtontransform (generic function with 1 method)

With the Newton transform, we can build Newton's method to find roots of g(x):

In [287]:
function Newtonmethod(g, guess)
    fixedpoint1(Newtontransform(g), guess)
end

Newtonmethod (generic function with 1 method)

The square root function can be expressed as the function obtained when you search for thzeros of the function:

>y &longrightarrow; y<sup>2</sup> - x so

>y<sup>2</sup> - x = 0 &iff;

>y<sup>2</sup> = x &iff;

>y = &Sqrt;x.

The zeros can be found using Newton's method:

In [288]:
function sqrtbyNM(x)
    Newtonmethod(y -> y^2 - x, 1.0)
end

sqrtbyNM (generic function with 1 method)

In [289]:
sqrtbyNM(144)

12.0

Here is a generalized function to find the fixed point of a transform of a function:

In [290]:
function fixedPointofTransform(g, transform, guess)
    fixedpoint1(transform(g), guess)
end

fixedPointofTransform (generic function with 1 method)

Let's recast the square root with this generalized function:

In [292]:
function sqrtbyGfpt(x)
    fixedPointofTransform(y -> x/y, damping, 1.0)
end

sqrtbyGfpt (generic function with 1 method)

In [293]:
sqrtbyGfpt(144)

12.0

Let's also recast square root using Newton's transform:

In [295]:
function sqrtNewt(x)
    fixedPointofTransform(y -> y^2 - x, Newtontransform, 1.0)
end

sqrtNewt (generic function with 1 method)

In [296]:
sqrtNewt(144)

12.0

In [297]:
sqrtNewt(9)

3.0